<h1>Imports</h1>

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from imblearn.combine import SMOTETomek


Using TensorFlow backend.


In [2]:
pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
RANDOM_SEED = 6

<h1>Dataset Importing</h1>

In [4]:
DATA_PATH = Path.cwd() / "../../datasets"

In [5]:
training_df = pd.read_csv(DATA_PATH / "train.csv", index_col="tripid")
training_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [6]:
training_df = training_df.drop('pickup_time', 1)
training_df = training_df.drop('drop_time', 1)

<h1>Data Preprocessing</h1>

In [7]:
features = training_df[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','fare']]
labels = training_df[['label']]
mapping = {'correct': 1, 'incorrect': 0}
labels = labels.replace({'label':mapping})

In [8]:
numerical_features = features.columns[features.dtypes != "object"].values

In [9]:
X_train, X_eval, y_train, y_eval = train_test_split(features, labels, test_size=0.3, shuffle=True, stratify= labels, random_state=RANDOM_SEED)

<h1>Data Cleaning</h1>

<h1>Feature Engineering</h1>

<h1>Model Training</h1>

In [10]:

## chain numerical preprocessing into a pipeline object
numeric_preprocessing_steps = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])


## create preprocessor stage of the final pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_preprocessing_steps, numerical_features)
    ],
    remainder = 'drop'
)

In [11]:
preprocessed_features = preprocessor.fit_transform(X_train)
preprocessed_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [12]:
preprocessed_features = preprocessor.fit_transform(X_eval)
preprocessed_test_features_data_frame = pd.DataFrame(data=preprocessed_features, columns=features.columns)

In [13]:
smk = SMOTETomek(random_state=42)
os_x_train, os_y_train = smk.fit_sample(preprocessed_features_data_frame,y_train)

In [14]:
os_y_train.shape

(21404, 1)

In [15]:
estimators = MultiOutputClassifier(
    estimator = RandomForestClassifier(random_state=10)
)

In [16]:
full_pipeline = Pipeline([
    ('estimators', estimators)
])

full_pipeline

Pipeline(memory=None,
         steps=[('estimators',
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                        max_depth=None,
                                                                        max_features='auto',
                                                                        max_leaf_nodes=None,
                                                                        max_samples=None,
                                                                        min_impurity_decrease=0.0,
                                                                        min_impurity_split=None,
                               

In [17]:
kfold = StratifiedKFold(n_splits=20,random_state=42,shuffle=False)
estimator =  RandomForestClassifier(max_depth=2, random_state=0)

score = cross_val_score(estimator,os_x_train, os_y_train, scoring="f1", cv=kfold,n_jobs=4)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [18]:
score

array([0.71043771, 0.72544081, 0.73349633, 0.72881356, 0.73631003,
       0.72147651, 0.74875208, 0.72047579, 0.74788494, 0.72560467,
       0.71893244, 0.73462511, 0.71152229, 0.72361809, 0.7228497 ,
       0.70386266, 0.73817568, 0.71367884, 0.71057192, 0.70496084])

In [19]:

## Train the model
estimator.fit(os_x_train, os_y_train)

# Predict for the evaluation set
print("Training Accuracy: %.2f" % (estimator.score(preprocessed_test_features_data_frame, y_eval)*100), "%")
y_pred = estimator.predict(preprocessed_test_features_data_frame)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Training Accuracy: 77.26 %


In [20]:
confusion_matrix = confusion_matrix(y_eval, y_pred)
confusion_matrix

array([[ 276,  228],
       [ 944, 3705]])

In [21]:
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

           0       0.23      0.55      0.32       504
           1       0.94      0.80      0.86      4649

    accuracy                           0.77      5153
   macro avg       0.58      0.67      0.59      5153
weighted avg       0.87      0.77      0.81      5153



In [22]:
estimator.feature_importances_

array([0.08998417, 0.28264761, 0.04420087, 0.00907859, 0.23881143,
       0.33527732])

<h1>Model Validation</h1>

In [23]:
test_set = pd.read_csv(DATA_PATH / "test.csv", index_col="tripid")
test_set.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [24]:
test_probs = estimator.predict(test_set[['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup','fare']])

In [25]:
submission_set = pd.read_csv(DATA_PATH / "sample_submission.csv", index_col="tripid")
submission_set.head()

submission_set['prediction']= test_probs

In [26]:
submission_set.to_csv('../../submissions/model6/teamCluster_submission_01.csv', index=True)
print("Completed!")

Completed!


In [ ]:
submission_set['prediction'].idxmin()

In [ ]:
submission_set['prediction'].value_counts()